In [258]:
with open('./data/input_14.txt') as fh:
    file_input = fh.read().strip()

In [101]:
test = """10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL"""

In [324]:
bon

{'KHKGT': 225360576,
 'QDVJ': 600961536,
 'DCFZ': 375600960,
 'PSHF': 225360576,
 'NZVS': 300480768}

In [269]:
test2 = """157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT"""

In [166]:
test3 = """2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF"""

In [242]:
test4 = """171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX"""

In [271]:
import re
import math
def parse(txt):
    return [list(zip([int(d) for d in re.findall('\d+', line)], re.findall('[a-zA-Z]+', line))) for line in txt.split('\n')]

def build_replacements(recipes):
    repl = {rec[-1][1]: (rec[-1][0], rec[:-1]) for rec in recipes}
    return repl

def find_replacement(key, num, repl):
    if key == 'ORE':
        return {'ORE': num}, {}
    req, el = repl[key]
    fac = math.ceil(num / req)
    rest = req * fac - num
    num_el = {e[1]: e[0]*fac for e in el}
#     print("repl: ", num, key, num_el, 'rest: ', key, rest)
    bonus = {key: rest} if rest else {}
#     print('fr ', num_el, bonus)
    return num_el, bonus

def update(d1, d2):
    for key in d2.keys():
        d1[key] = d1.get(key, 0) + d2[key]
    return d1

def subtract(d1, d2):
    keys = list(d2.keys())
    for key in keys:
        if key in d1.keys():
            if d2[key] < d1[key]:
                d1[key] -= d2.pop(key)
            elif d2[key] > d1[key]:
                d2[key] -= d1.pop(key)
            else:
                d1.pop(key)
                d2.pop(key)
                
    return d1, d2

In [476]:
def calculate(txt, bonus = {}, fuel=1):
    recipes = parse(txt)
    repl = build_replacements(recipes)

    store = {'FUEL': fuel}
    while store.keys() != set(['ORE']) and len(store) > 0:
    #     print('step')
        new_store = {}
        for key in store.keys():
            replacement, new_bonus = find_replacement(key, store[key], repl)
            new_store = update(new_store, replacement)
            bonus = update(bonus, new_bonus)
            new_store, bonus = subtract(new_store, bonus)
#             print('!!!', new_store, bonus)
        store = new_store
#         print(len(store), "!")
    if 'ORE' not in store:
        store['ORE'] = 0
    return (store, bonus)

In [477]:
calculate(test)

({'ORE': 31}, {'A': 2})

In [294]:
calculate(test2, {})

({'ORE': 13312}, {'KHKGT': 3, 'QDVJ': 8, 'DCFZ': 5, 'PSHF': 3, 'NZVS': 4})

In [167]:
calculate(test3)

({'ORE': 180697},
 {'GNMV': 5,
  'VPVL': 3,
  'RFSQX': 3,
  'NVRVD': 1,
  'JNWZP': 6,
  'VJHF': 3,
  'MNCFX': 2})

In [237]:
# part 1
ore, left = calculate(file_input)
ore

{'ORE': 399063}

In [193]:
# part 2
total_cargo = 1000000000000

In [506]:
ore, left = calculate(test4, bonus={}, fuel=460664)
print(ore['ORE'], len(left))

999998346916 11


In [488]:
ore, left = calculate(file_input, bonus={}, fuel=460665)
print(ore['ORE'], len(left))

109274881430 34


In [505]:
# part 2
_high = 100000000
_low = 1
while (_high - _low) > 1:
    i = _low + (_high - _low) // 2
    ore, left = calculate(file_input, bonus={}, fuel=i)
    if ore['ORE'] > total_cargo:
        _high = i
    else:
        _low = i
    
    print(ore['ORE'], i)

11860552551838 50000000
5930276331168 25000000
2965138300177 12500000
1482569190843 6250000
741284745758 3125000
1111926852107 4687500
926605824271 3906250
1019266451971 4296875
972935957032 4101562
996101055779 4199218
1007683600574 4248046
1001892317191 4223632
998996678626 4211425
1000444458413 4217528
999720429392 4214476
1000082305792 4216002
999901466356 4215239
999991871555 4215620
1000037102236 4215811
1000014338746 4215715
1000002928316 4215667
999997255494 4215643
1000000090514 4215655
999998667331 4215649
999999411662 4215652
999999677090 4215653
999999749382 4215654


In [507]:
calculate(file_input, bonus={}, fuel=4215654)

({'ORE': 999999749382},
 {'KLJMS': 6,
  'KZDJ': 2,
  'DLZS': 6,
  'VCFX': 4,
  'JVMF': 6,
  'PLSCB': 3,
  'JLVQ': 6,
  'LPRL': 1,
  'MZQLG': 1,
  'JNWJ': 1,
  'PCSM': 2,
  'PJFBD': 7,
  'KBWF': 3,
  'PRMJ': 1,
  'QDZB': 4,
  'WDKW': 2,
  'ZMFXQ': 5,
  'TLRST': 3,
  'LHWZQ': 1,
  'LWNKB': 1,
  'RPLCK': 3,
  'XNWV': 5,
  'HTZW': 3,
  'FGVL': 6,
  'FZDP': 6,
  'JBPHS': 3,
  'CRPGK': 2,
  'GJHX': 3,
  'TKVCX': 4,
  'VFZX': 5,
  'ZBLC': 2,
  'NRSD': 1,
  'STHQ': 1,
  'TKRT': 5,
  'DQFL': 5,
  'VNMV': 4,
  'CNLFV': 4,
  'VFTG': 1})

In [464]:
# not needed in the end

def find_period(txt, cargo = total_cargo, bonus = {}):

    fuel = 0
    seen = {}
    while cargo > 0:
        ore, bonus = calculate(txt, bonus)
        cargo -= ore['ORE']
        if cargo < 0:
            return (fuel, cargo)
        fuel += 1
        _hash = tuple(sorted(bonus.items()))
        if _hash in seen:
            break
        seen[_hash] = (fuel, cargo)
        #print(cargo, fuel, bonus)
#         print(ore['ORE'])
    return (fuel, cargo, seen)


def without_rest(txt, cargo = total_cargo, bonus={}):
    bonus = {}
    fuel = 0
    while cargo > 0:
        ore, bonus = calculate(txt, bonus)
        cargo -= ore['ORE']
        if cargo < 0:
            return (fuel, cargo)
        fuel += 1
        if sum(bonus.values()) == 0:
            break
    return (fuel, cargo)

def recycle(txt, cargo, bonus={}):
    fuel = 0
    while cargo > 0:
        ore, bonus = calculate(txt, bonus)
        cargo -= ore['ORE']
        if cargo < 0:
            return (fuel, cargo, bonus)
        fuel += 1
#         print(cargo, bonus)
    return (fuel, cargo, bonus)